In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
# Import Dataset
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [4]:
# Convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [5]:
# simple_preprocess -> Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, 
# that won’t be processed any further.

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 's', 'thing', 'car', 'nntp_poste', 'host', 'umd', 'organization', 'university', 'maryland_college', 'park', 'line', 'wonder', 'anyone', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'front_bumper', 'separate', 'rest', 'body', 'know', 'anyone', 'tellme', 'model', 'name', 'engine', 'specs', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
# doc2bow() simply counts the number of occurrences of each distinct word, converts the word to its
# integer word id and returns the result as a sparse vector. 
# The sparse vector [(0, 1), (1, 1)] therefore reads: in the document “Human computer interaction”, 
# the words computer (id 0) and human (id 1) appear once; the other ten dictionary words appear 
# (implicitly) zero times.
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 5), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1)]]


In [10]:
id2word[0]

'addition'

In [11]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('anyone', 2),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('front_bumper', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('maryland_college', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('organization', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('specs', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('umd', 1),
  ('university', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [12]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [17]:
print(lda_model.print_topics())

[(0, '0.072*"clipper" + 0.067*"algorithm" + 0.052*"clipper_chip" + 0.051*"rsa" + 0.033*"crypto" + 0.024*"export" + 0.020*"white_house" + 0.020*"specification" + 0.019*"device" + 0.017*"enable"'), (1, '0.075*"god" + 0.040*"evidence" + 0.037*"christian" + 0.027*"reason" + 0.024*"believe" + 0.023*"faith" + 0.019*"bible" + 0.019*"sense" + 0.017*"religion" + 0.015*"exist"'), (2, '0.101*"test" + 0.068*"server" + 0.045*"motif" + 0.030*"resource" + 0.026*"workstation" + 0.021*"ron" + 0.019*"roger" + 0.019*"jr" + 0.017*"last_night" + 0.016*"modify"'), (3, '0.113*"space" + 0.031*"earth" + 0.028*"launch" + 0.026*"moon" + 0.025*"mission" + 0.024*"orbit" + 0.023*"nasa" + 0.020*"satellite" + 0.018*"mar" + 0.016*"rocket"'), (4, '0.052*"armenian" + 0.040*"greek" + 0.038*"village" + 0.032*"turk" + 0.030*"soldier" + 0.027*"turkish" + 0.021*"turkey" + 0.018*"cub" + 0.013*"inhabitant" + 0.013*"km"'), (5, '0.067*"player" + 0.045*"season" + 0.035*"brain" + 0.034*"nhl" + 0.033*"scott" + 0.032*"chicago" + 0.0

In [22]:
show_topics(num_topics=10, num_words=10, log=False, formatted=True)

NameError: name 'show_topics' is not defined

In [21]:
for i in range(0, lda_model.num_topics-1):
    print(lda_model.print_topic(i))


0.072*"clipper" + 0.067*"algorithm" + 0.052*"clipper_chip" + 0.051*"rsa" + 0.033*"crypto" + 0.024*"export" + 0.020*"white_house" + 0.020*"specification" + 0.019*"device" + 0.017*"enable"
0.075*"god" + 0.040*"evidence" + 0.037*"christian" + 0.027*"reason" + 0.024*"believe" + 0.023*"faith" + 0.019*"bible" + 0.019*"sense" + 0.017*"religion" + 0.015*"exist"
0.101*"test" + 0.068*"server" + 0.045*"motif" + 0.030*"resource" + 0.026*"workstation" + 0.021*"ron" + 0.019*"roger" + 0.019*"jr" + 0.017*"last_night" + 0.016*"modify"
0.113*"space" + 0.031*"earth" + 0.028*"launch" + 0.026*"moon" + 0.025*"mission" + 0.024*"orbit" + 0.023*"nasa" + 0.020*"satellite" + 0.018*"mar" + 0.016*"rocket"
0.052*"armenian" + 0.040*"greek" + 0.038*"village" + 0.032*"turk" + 0.030*"soldier" + 0.027*"turkish" + 0.021*"turkey" + 0.018*"cub" + 0.013*"inhabitant" + 0.013*"km"
0.067*"player" + 0.045*"season" + 0.035*"brain" + 0.034*"nhl" + 0.033*"scott" + 0.032*"chicago" + 0.028*"boston" + 0.026*"pen" + 0.022*"trade" + 0.

In [13]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.072*"clipper" + 0.067*"algorithm" + 0.052*"clipper_chip" + 0.051*"rsa" + '
  '0.033*"crypto" + 0.024*"export" + 0.020*"white_house" + '
  '0.020*"specification" + 0.019*"device" + 0.017*"enable"'),
 (1,
  '0.075*"god" + 0.040*"evidence" + 0.037*"christian" + 0.027*"reason" + '
  '0.024*"believe" + 0.023*"faith" + 0.019*"bible" + 0.019*"sense" + '
  '0.017*"religion" + 0.015*"exist"'),
 (2,
  '0.101*"test" + 0.068*"server" + 0.045*"motif" + 0.030*"resource" + '
  '0.026*"workstation" + 0.021*"ron" + 0.019*"roger" + 0.019*"jr" + '
  '0.017*"last_night" + 0.016*"modify"'),
 (3,
  '0.113*"space" + 0.031*"earth" + 0.028*"launch" + 0.026*"moon" + '
  '0.025*"mission" + 0.024*"orbit" + 0.023*"nasa" + 0.020*"satellite" + '
  '0.018*"mar" + 0.016*"rocket"'),
 (4,
  '0.052*"armenian" + 0.040*"greek" + 0.038*"village" + 0.032*"turk" + '
  '0.030*"soldier" + 0.027*"turkish" + 0.021*"turkey" + 0.018*"cub" + '
  '0.013*"inhabitant" + 0.013*"km"'),
 (5,
  '0.067*"player" + 0.045*"season" + 

In [14]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -14.176559763680439

Coherence Score:  0.5163890312318851


In [23]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/deepshikha/.local/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
13    -0.379783  0.008571       1        1  33.826191
10    -0.351080 -0.036524       2        1  28.056835
16    -0.318754  0.160001       3        1  10.857606
14    -0.209900 -0.292712       4        1   7.540839
8      0.022399 -0.042695       5        1   4.648159
1     -0.223054  0.246557       6        1   3.075295
17    -0.009532 -0.210048       7        1   2.136385
18    -0.005169 -0.098735       8        1   1.656911
15     0.074401 -0.000666       9        1   1.410655
3      0.072080  0.017076      10        1   1.020934
4      0.073950  0.058024      11        1   1.003426
5      0.105719  0.011050      12        1   0.951477
11     0.087814  0.009344      13        1   0.869672
12     0.134442  0.018817      14        1   0.580911
2      0.148622  0.024765      15        1   0.578891
6      0.158269  0.025703      16        1   0.406067
9      0.149145  0.026365      17        1   0.388500
7      0.155069  0.025191      18        1   0.385798
19     0.153322  0.026880      19        1   0.319311
0      0.162041  0.023035      20        1   0.286143, topic_info=      Category          Freq     Term         Total  loglift  logprob
5533   Default  51911.000000       ax  51911.000000  30.0000  30.0000
80     Default  22376.000000      not  22376.000000  29.0000  29.0000
63     Default  14055.000000       do  14055.000000  28.0000  28.0000
48     Default   5094.000000     year   5094.000000  27.0000  27.0000
21     Default  19601.000000     line  19601.000000  26.0000  26.0000
...        ...           ...      ...           ...      ...      ...
14098  Topic20      0.035222      pkp      1.242019   2.2936 -11.7656
1549   Topic20      6.591735    voice    522.455444   1.4837  -6.5337
4409   Topic20      1.785752  approve    206.332016   1.1068  -7.8397
786    Topic20      3.624154  develop    576.583130   0.7869  -7.1319
7995   Topic20      0.035223   jumper      1.242149   2.2935 -11.7656

[832 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1051     18  0.980103              ab
590      17  0.988142        absolute
3220     17  0.923188  absolute_truth
415      18  0.950375          absurd
1630      2  0.271452          accept
...     ...       ...             ...
48        1  0.337015            year
48        3  0.165269            year
48        4  0.240052            year
48        8  0.251044            year
48       10  0.006477            year

[1074 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[14, 11, 17, 15, 9, 2, 18, 19, 16, 4, 5, 6, 12, 13, 3, 7, 10, 8, 20, 1])